In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [35]:
df = pd.read_csv("data23aug23uhr.csv")

In [36]:
df.columns

Index(['participant.id_in_session', 'participant.code', 'participant.label',
       'participant._is_bot', 'participant._index_in_pages',
       'participant._max_page_index', 'participant._current_app_name',
       'participant._current_page_name', 'participant.time_started',
       'participant.visited',
       ...
       'bms_experiment.2.player.q4_a2', 'bms_experiment.2.player.q4_a3',
       'bms_experiment.2.player.q4_a4', 'bms_experiment.2.player.q4_a5',
       'bms_experiment.2.player.attentive_1',
       'bms_experiment.2.player.attentive_2',
       'bms_experiment.2.player.finished', 'bms_experiment.2.player.payoff',
       'bms_experiment.2.group.id_in_subsession',
       'bms_experiment.2.subsession.round_number'],
      dtype='object', length=110)

In [37]:
df.drop(["participant._is_bot","participant.code","participant.label",'participant._index_in_pages',
       'participant._max_page_index', 'participant._current_app_name',
       'participant._current_page_name', 'participant.time_started',
       'participant.visited','participant.mturk_worker_id',
       'participant.mturk_assignment_id', 'participant.payoff',
       'participant.payoff_plus_participation_fee', 'session.code',
       'session.label', 'session.mturk_HITId', 'session.mturk_HITGroupId',
       'session.comment', 'session.is_demo',
       'session.config.participation_fee',
       'session.config.real_world_currency_per_point','bms_experiment.1.player.payoff','bms_experiment.2.player.payoff','bms_experiment.1.player.id_in_group',
       'bms_experiment.1.player.trans_cond','bms_experiment.1.group.id_in_subsession',
       'bms_experiment.1.subsession.round_number',
       'bms_experiment.2.player.id_in_group',
       'bms_experiment.2.player.trans_cond','bms_experiment.2.group.id_in_subsession',
       'bms_experiment.2.subsession.round_number'], axis=1, inplace=True)

In [38]:
df.columns

Index(['participant.id_in_session', 'bms_experiment.1.player.age',
       'bms_experiment.1.player.gender', 'bms_experiment.1.player.installed',
       'bms_experiment.1.player.competence',
       'bms_experiment.1.player.competence_neg',
       'bms_experiment.1.player.benevolence',
       'bms_experiment.1.player.benevolence_neg',
       'bms_experiment.1.player.no_central_entity',
       'bms_experiment.1.player.no_central_entity_neg',
       'bms_experiment.1.player.anonymity',
       'bms_experiment.1.player.anonymity_neg',
       'bms_experiment.1.player.no_tracking',
       'bms_experiment.1.player.no_tracking_neg',
       'bms_experiment.1.player.unlinkabilty',
       'bms_experiment.1.player.unlinkabilty_neg',
       'bms_experiment.1.player.understanding',
       'bms_experiment.1.player.q1_a1', 'bms_experiment.1.player.q1_a2',
       'bms_experiment.1.player.q1_a3', 'bms_experiment.1.player.q1_a4',
       'bms_experiment.1.player.q1_a5', 'bms_experiment.1.player.q2_a1',
    

In [39]:
df['q1_r1']= 0
df['q2_r1']= 0
df['q3_r1']= 0
df['q4_r1']= 0
df['q1_r2']= 0
df['q2_r2']= 0
df['q3_r2']= 0
df['q4_r2']= 0

In [40]:
solutions = [[0,0,1,0],[0,0,0,1],[1,0,0,1],[0,0,0,1]]

for roundnumber in range(1,3):
    for question in range(1,5):
        for i in df.T:
            question_score = 0
            if df['bms_experiment.{0}.player.q{1}_a5'.format(roundnumber,question)][i] == 1: #i am not sure direkt 0
                question_score = 0
            else:
                for answer in range(1,5):
                    if df['bms_experiment.{0}.player.q{1}_a{2}'.format(roundnumber,question,answer)][i] == solutions[question-1][answer-1]:
                        question_score = question_score + 1 
            df['q{0}_r{1}'.format(question,roundnumber)][i]=question_score

/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [41]:
df.drop(['bms_experiment.1.player.q1_a1', 'bms_experiment.1.player.q1_a2',
       'bms_experiment.1.player.q1_a3', 'bms_experiment.1.player.q1_a4',
       'bms_experiment.1.player.q1_a5', 'bms_experiment.1.player.q2_a1',
       'bms_experiment.1.player.q2_a2', 'bms_experiment.1.player.q2_a3',
       'bms_experiment.1.player.q2_a4', 'bms_experiment.1.player.q2_a5',
       'bms_experiment.1.player.q3_a1', 'bms_experiment.1.player.q3_a2',
       'bms_experiment.1.player.q3_a3', 'bms_experiment.1.player.q3_a4',
       'bms_experiment.1.player.q3_a5', 'bms_experiment.1.player.q4_a1',
       'bms_experiment.1.player.q4_a2', 'bms_experiment.1.player.q4_a3',
       'bms_experiment.1.player.q4_a4', 'bms_experiment.1.player.q4_a5',
       'bms_experiment.2.player.q1_a1', 'bms_experiment.2.player.q1_a2',
       'bms_experiment.2.player.q1_a3', 'bms_experiment.2.player.q1_a4',
       'bms_experiment.2.player.q1_a5', 'bms_experiment.2.player.q2_a1',
       'bms_experiment.2.player.q2_a2', 'bms_experiment.2.player.q2_a3',
       'bms_experiment.2.player.q2_a4', 'bms_experiment.2.player.q2_a5',
       'bms_experiment.2.player.q3_a1', 'bms_experiment.2.player.q3_a2',
       'bms_experiment.2.player.q3_a3', 'bms_experiment.2.player.q3_a4',
       'bms_experiment.2.player.q3_a5', 'bms_experiment.2.player.q4_a1',
       'bms_experiment.2.player.q4_a2', 'bms_experiment.2.player.q4_a3',
       'bms_experiment.2.player.q4_a4', 'bms_experiment.2.player.q4_a5',], axis=1, inplace=True)
df.columns

Index(['participant.id_in_session', 'bms_experiment.1.player.age',
       'bms_experiment.1.player.gender', 'bms_experiment.1.player.installed',
       'bms_experiment.1.player.competence',
       'bms_experiment.1.player.competence_neg',
       'bms_experiment.1.player.benevolence',
       'bms_experiment.1.player.benevolence_neg',
       'bms_experiment.1.player.no_central_entity',
       'bms_experiment.1.player.no_central_entity_neg',
       'bms_experiment.1.player.anonymity',
       'bms_experiment.1.player.anonymity_neg',
       'bms_experiment.1.player.no_tracking',
       'bms_experiment.1.player.no_tracking_neg',
       'bms_experiment.1.player.unlinkabilty',
       'bms_experiment.1.player.unlinkabilty_neg',
       'bms_experiment.1.player.understanding',
       'bms_experiment.1.player.attentive_1',
       'bms_experiment.1.player.attentive_2',
       'bms_experiment.1.player.finished', 'bms_experiment.2.player.age',
       'bms_experiment.2.player.gender', 'bms_experiment.

In [ ]:
df['invertedCheck']= True
for roundnumber in range(1,3):
    for trustAspect in ['competence','benevolence','no_central_entity','anonymity','no_tracking','unlinkabilty']:
        if ((((df['bms_experiment.{0}.player.{1}'.format(roundnumber,trustAspect)][1]+df['bms_experiment.{0}.player.{1}_neg'.format(roundnumber,trustAspect)][1])/2) < 2) | (((df['bms_experiment.{0}.player.{1}'.format(roundnumber,trustAspect)][1]+df['bms_experiment.{0}.player.{1}_neg'.format(roundnumber,trustAspect)][1])/2) > 5))

In [9]:
q1 = 0
if df['bms_experiment.1.player.q1_a5'][4] == 1:
        q1 = 0
else:
        if df['bms_experiment.1.player.q1_a1'][4] == 0:
            q1 = q1 + 1
        if df['bms_experiment.1.player.q1_a2'][4] == 0:
            q1 = q1 + 1
        if df['bms_experiment.1.player.q1_a3'][4] == 1:
            q1 = q1 + 1
        if df['bms_experiment.1.player.q1_a4'][4] == 0:
            q1 = q1 + 1
df['q1_score'][i]=q1
print(df['q1_score'][4])

3


/Users/julia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [34]:
df['participant.id_in_session']

0        1
1        2
2        3
3        4
4        5
      ... 
295    296
296    297
297    298
298    299
299    300
Name: participant.id_in_session, Length: 300, dtype: int64

In [48]:
df['q4_r1']

0      0
1      4
2      0
3      0
4      1
      ..
295    0
296    0
297    0
298    0
299    0
Name: q4_r1, Length: 300, dtype: int64